In [1]:
!pip install -q pyMuPDF tqdm accelerate bitsandbytes flash-attn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 40.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 11.7 MB/s eta 0:00:00


In [2]:
!pip install torch torchvision torchaudio
!pip install -U transformers sentence-transformers

In [3]:
# Download PDF file
import os
import requests

pdf_path = "human-nutrition-text.pdf"

if not os.path.exists(pdf_path):
  print("FIle dosent exist, downloading...")

  # The URL of the PDF you want to download
  url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"
  filename = pdf_path

  response = requests.get(url)

  if response.status_code == 200:
    with open(filename, "wb") as file:
      file.write(response.content)
    print(f"The file has been downloaded and saved as {filename}")
  else:
    print(f"Failed to download the file. Status code: {response.status_code}")

else:
  print(f"File {pdf_path} exists")

FIle dosent exist, downloading...
The file has been downloaded and saved as human-nutrition-text.pdf


In [4]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
  cleaned_text = text.replace("\n"," ").strip()
  return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:

  doc = fitz.open(pdf_path)
  pages_and_texts = []
  for page_number, page in tqdm(enumerate(doc)):
    text = page.get_text()
    text = text_formatter(text)
    pages_and_texts.append({"page_number":page_number - 41, # adjust page numbers since our PDF starts from page 42
                            "page_chart_count": len(text),
                            "page_word_count": len(text.split(" ")),
                            "page_sentence_count_raw": len(text.split(". ")),
                            "page_token_count": len(text) / 4, # 1 token = ~4 chars
                            "text": text
                            })
  return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path = pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': -41,
  'page_chart_count': 29,
  'page_word_count': 4,
  'page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_chart_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''}]

In [5]:
pages_and_texts[41]

{'page_number': 0,
 'page_chart_count': 762,
 'page_word_count': 133,
 'page_sentence_count_raw': 4,
 'page_token_count': 190.5,
 'text': 'Special Thanks to  Bill Chismar – University of Hawai’i at Mānoa, Dean of Outreach  College  The Children’s Healthy Living Summer Institute – University of  Hawai’i at Mānoa  Open Educational Resources  This text is provided to you as an Open Educational Resource (OER)  which you access online. It is designed to give you a comprehensive  introduction to human nutrition at no or very nominal cost. \xa0It  contains both written and graphic text material, intra-text links to  other internal material which may aid in understanding topics and  concepts, intra-text links to the appendices and glossary for tables  and definitions of words, and extra-text links to videos and web  material that clarifies and augments topics and concepts.  xlii  |  Acknowledgements'}

In [6]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df

,page_number,page_chart_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,0.00,
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...
...,...,...,...,...,...,...
1203,1162,1676,252,18,419.00,39. Exercise 10.2 & 11.3 reused “Egg Oval Food...
1204,1163,1617,254,10,404.25,Images / Pixabay License; “Pumpkin Cartoon Ora...
1205,1164,1715,261,13,428.75,Flashcard Images Note: Most images in the fla...
1206,1165,1733,268,13,433.25,ShareAlike 11. Organs reused “Pancreas Organ ...


In [7]:
df.describe().round(2)

,page_number,page_chart_count,page_word_count,page_sentence_count_raw,page_token_count
count,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.00,198.30,9.97,287.00
std,348.86,560.38,95.76,6.19,140.10
min,-41.00,0.00,1.00,1.00,0.00
25%,260.75,762.00,134.00,4.00,190.50
50%,562.50,1231.50,214.50,10.00,307.88
75%,864.25,1603.50,271.00,14.00,400.88
max,1166.00,2308.00,429.00,32.00,577.00


In [8]:
def chunk_text(text: str, chunk_size: int = 500) -> list:
  chunks = []
  current_chunk = ""
  words = text.split()

  for word in words:
    if len(current_chunk) + len(word) +1 <= chunk_size:
      current_chunk += word + " "
    else:
      chunks.append(current_chunk.strip())
      current_chunk = word + " "

  if current_chunk:
    chunks.append(current_chunk.strip())
  return chunks


def chunk_pdf_pages(paegs_and_texts: list, chunk_size: int = 500) -> list[dict]:
  all_chunks = []
  for page in pages_and_texts:
    page_number = page['page_number']
    page_text = page['text']

    chunks = chunk_text(text = page_text, chunk_size = chunk_size)
    for i, chunk in enumerate(chunks):
      all_chunks.append({
          "page_number": page_number,
          "chunk_index": i,
          "chunk_char_count":len(chunk),
          "chunk_word_count": len(chunk.split()),
          "chunk_token_count": len(chunk) / 4,
          "chunk_text": chunk
      })
  return all_chunks


chunked_pages = chunk_pdf_pages(pages_and_texts, chunk_size=500)
print(f"Total Chunks: {len(chunked_pages)}")
print(f"First chunk (page {chunked_pages[0]['page_number']}): {chunked_pages[0]['chunk_text'][:200]}...")

Total Chunks: 3321
First chunk (page -41): Human Nutrition: 2020 Edition...


In [9]:
chunked_pages[201]

{'page_number': 49,
 'chunk_index': 2,
 'chunk_char_count': 494,
 'chunk_word_count': 83,
 'chunk_token_count': 123.5,
 'chunk_text': 'subjects of this study? Humans or animals? If human, are any traits/characteristics noted? You may realize you have more in common with certain program participants and can use that as a basis to gauge if the study applies to you. 4. Credible reports often disseminate new findings in the context of previous research. A single study on its own gives you very limited information, but if a body of literature supports a finding, it adds to credibility. 5. Peer-reviewed articles deliver a broad'}

## Chunking Strategy 2: Semantic Chunking

In [ ]:
!pip install nltk

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
nltk.download('punkt', quiet=True)

semantic_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def semantic_chunk_text(text: str, similarity_threshold: float = 0.8, max_tokens: int = 500) -> list:
  sentences = nltk.sent_tokenize(text, language="english")

  if not sentences:
    return []

  embeddings = semantic_model.encode(sentences)
  chunks = []
  current_chunk = [sentences[0]]
  current_embedding = embeddings[0]
  for i in range(1, len(sentences)):
    sim = cosine_similarity([current_embedding],[embeddings[i]])[0][0]
    chunk_token_count = len(" ".join(current_chunk)) // 4

    if sim >= similarity_threshold and chunk_token_count < max_tokens:

      current_chunk.append(sentences[i])
      current_embedding = np.mean([current_embedding, embeddings[i]], axis=0)

    else:
      chunks.append(" ".join(current_chunk))
      current_chunk = [sentences[i]]
      current_embedding = embeddings[i]

  if current_chunk:
    chunks.append(" ".join(current_chunk))
  return chunks


from tqdm.auto import tqdm

def semantic_chunk_pdf_pages(pages_and_texts: list, similarity_threshold: float = 0.8, max_tokens: int = 500) -> list[dict]:

  all_chunks = []

  for page in tqdm(pages_and_texts, desc="Semantic chunking pages"):

    page_number = page['page_number']
    page_text = page['text']

    chunks = semantic_chunk_text(page_text, similarity_threshold=similarity_threshold, max_tokens=max_tokens)

    for i, chunk in enumerate(chunks):
      all_chunks.append({
          "page_number":page_number,
          "chunk_index":i,
          "chunk_char_count": len(chunk),
          "chunk_word_count": len(chunk.split()),
          "chunk_token_count": len(chunk) / 4,
          "chunk_text": chunk
      })
  return all_chunks



In [ ]:
semantic_chunk_text("This is a test sentence. This is another test")

['This is a test sentence.', 'This is another test']

In [ ]:
nltk.download('punkt_tab')
semantic_chunked_pages = semantic_chunk_pdf_pages(pages_and_texts,
                                                  similarity_threshold=0.75,
                                                  max_tokens=500)

print(f"Total Chunks: {len(semantic_chunked_pages)}")

print(f"First chunk (page {semantic_chunked_pages[0]['page_number']}): {semantic_chunked_pages[0]['chunk_text'][:200]}...")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Semantic chunking pages:   0%|          | 0/1208 [00:00<?, ?it/s]

Total Chunks: 12016
First chunk (page -41): Human Nutrition: 2020 Edition...


## Chunking Strategy 3: Recursive Chunking

In [ ]:
def recursive_chunk_text(text: str, max_chunk_size: int = 1000, min_chunk_size: int = 100) -> list:

  def split_chunk(chunk: str) -> list:
    if len(chunk) <= max_chunk_size:
      return [chunk]

    sections = chunk.split("\n\n")
    if len(sections) > 1:
      result = []
      for section in sections:
        if section.strip():
          result.extend(split_chunk(section.strip()))
      return result

    sections = chunk.split("\n")
    if len(sections) > 1:
      result = []
      for section in sections:
        if section.strip():
          result.extend(split_chunk(section.strip()))
      return result

    sentences = nltk.sent_tokenize(chunk)
    chunks, current_chunk, current_size = [], [], 0

    for sentence in sentences:
      if current_size + len(sentence) > max_chunk_size:
        if current_chunk:
          chunks.append(" ".join(current_chunk))
        current_chunk = [sentence]
        current_size = len(sentence)

      else:
        current_chunk.append(sentence)
        current_size += len(sentence)

    if current_chunk:
      chunks.append(" ".join(current_chunk))

    return chunks

  return split_chunk(text)


def recursive_chunk_pdf_Pages(pages_and_texts: list,
                              max_chunk_size: int = 1000,
                              min_chunk_size: int = 100) -> list[dict]:

  all_chunks = []

  for page in tqdm(pages_and_texts, desc = "Recursive chunking pages"):
    page_number = page['page_number']
    page_text = page['text']

    chunks = recursive_chunk_text(page_text,max_chunk_size, min_chunk_size)

    for i, chunk in enumerate(chunks):
      all_chunks.append({
          'page_number': page_number,
          'chunk_index': i,
          'chunk_char_count': len(chunk),
          'chunk_word_count': len(chunk.split()),
          'chunk_token_count': len(chunk) / 4,
          "chunk_text":chunk
      })

  return all_chunks


In [ ]:
recursive_chunked_pages = recursive_chunk_pdf_Pages(pages_and_texts,
                                                      max_chunk_size=1000,
                                                      min_chunk_size=100)

print(f"Total Chunks: {len(recursive_chunked_pages)}")
print(f"First chunk (page {recursive_chunked_pages[0]['page_number']}): {recursive_chunked_pages[0]['chunk_text'][:200]}...")

Recursive chunking pages:   0%|          | 0/1208 [00:00<?, ?it/s]

Total Chunks: 2010
First chunk (page -41): Human Nutrition: 2020 Edition...


## Chunking Strategy 4: Document Structure Based chunking

In [ ]:
a = "test text"
a.start()

AttributeError: 'str' object has no attribute 'start'

In [ ]:
import re
import random
import textwrap

def _is_chapter_header_page(text: str) -> bool:

  return re.search("university\s+of\s+hawai", text, flags=re.IGNORECASE) is not None

def _guess_title_from_page(text: str) -> str:

  m = re.search("university\s+of\s+hawai", text, flags=re.IGNORECASE)
  if m:
    title = text[:m.start()].strip()
    title = re.sub(r"\s+"," ",title).strip()
    if 10 <= len(title) <= 180:
      return title

  t = re.sub(r"\s+"," ",text).strip()
  return t[:120] if t else "Untitled Chapter"

def chapter_chunk_pdf_pages(pages_and_texts: list[dict]) -> list[dict]:

  if not pages_and_texts:
    return []

  #Find all page indices that look like the start of a chapter
  chapter_starts = []
  for i, p in enumerate(pages_and_texts):
    txt = p['text']
    if _is_chapter_header_page(txt):
      chapter_starts.append(i)

  if not chapter_starts:
    # Treat entire doc as one chapter
    all_text = " ".join(p['text'] for p in pages_and_texts).strip()
    return [{
        "chapter_index": 0,
        "title": _guess_title_from_page(pages_and_texts[0]['text']),
        "page_start": pages_and_texts[0]['page_number'],
        "page_end": pages_and_texts[-1]['page_number'],
        "chunk_char_count": len(all_text),
        "chunk_word_count": len(all_text.split()),
        "chunk_token_count": round(len(all_text) / 4, 2),
        "chunk_text": all_text

    }]

  chapter_chunks = []
  for ci, s in enumerate(chapter_starts):
    e = (chapter_starts[ci+1] -1) if (ci + 1 < len(chapter_starts)) else (len(pages_and_texts)-1)

    if e < s:
      continue

    pages = pages_and_texts[s: e + 1]
    text_contact = " ".join(p['text'] for p in pages).strip()
    title = _guess_title_from_page(pages[0]['text'])

    chapter_chunks.append({
        "chapter_index": ci,
        "title": title,
        "page_start": pages[0]['page_number'],
        "page_end": pages[-1]['page_number'],
        "chunk_char_count": len(text_contact),
        "chunk_word_count": len(text_contact.split()),
        "chunk_token_count": round(len(text_contact) / 4, 2),
        "chunk_text": text_contact
    })
  return chapter_chunks

<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-464714505.py:7: SyntaxWarning: invalid escape sequence '\s'
  return re.search("university\s+of\s+hawai", text, flags=re.IGNORECASE) is not None
/tmp/ipython-input-464714505.py:11: SyntaxWarning: invalid escape sequence '\s'
  m = re.search("university\s+of\s+hawai", text, flags=re.IGNORECASE)


In [ ]:
structure_chunked_pages = chapter_chunk_pdf_pages(pages_and_texts)

print(f"Total chapter-based chunks: {len(structure_chunked_pages)}")
if structure_chunked_pages:
  first = structure_chunked_pages[0]
  print(f"First chapter (page {first['page_start']} - {first['page_end']}): {first['chunk_text'][:200]}...")
else:
  print("No chapters detected")

Total chapter-based chunks: 171
First chapter (page -39 - -39): Human Nutrition: 2020  Edition  UNIVERSITY OF HAWAI‘I AT MĀNOA  FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM  ALAN TITCHENAL, SKYLAR HARA,  NOEMI ARCEO CAACBAY, WILLIAM  MEINKE-LAU, YA-YUN YANG, MARIE  K...


## Strategy 5: LLM based Chunking

In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")

In [ ]:
from openai import OpenAI
from typing import List, Dict
from tqdm.auto import tqdm

client = OpenAI()

def llm_based_chunk(text: str, chunk_size: int = 1000, model: str = "gpt-4o-mini") -> List[str]:

  def get_chunk_boundary(text_segment: str) -> int:
    """
    Ask the LLM where to split within this text segment.
    Returns an index (int) within text_segment.
    """
    prompt = f"""
    Analyze the following text and identify the best point to split it
    into two semantically coherent parts.
    The split should occur near {chunk_size} characters.

    Text:
    \"\"\"{text_segment}\"\"\"

    Return only the integer index (character position) within this text
    where the split should occur. Do not return any explanation.
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role":"system","content":"You are a text analysis expert."},
            {"role":"user","content":prompt}
        ],
        temperature = 0
    )


    split_str = response.choices[0].message.content.strip()

    try:
      split_point = int(split_str)

    except ValueError:
      split_point = chunk_size
    return split_point

  chunks = []
  remaining_text = text

  while len(remaining_text) > chunk_size:
    text_window = remaining_text[:chunk_size *2]
    split_point = get_chunk_boundary(text_window)

    if split_point < 100 or split_point > len(text_window) - 100:
      split_point = chunk_size

    chunks.append(remaining_text[:split_point].strip())
    remaining_text = remaining_text[split_point:].strip()

  if remaining_text:
    chunks.append(remaining_text)
  return chunks

def llm_based_chunk_pdf_pages(pages_and_texts: list[dict], chunk_size: int = 1000, model: str = "gpt-4o-mini"):
  all_chunks = []

  for page in tqdm(pages_and_texts, desc = "LLM-based chunking pages"):
    page_number = page['page_number']
    page_text = page['text']

    chunks = llm_based_chunk(page_text, chunk_size, model)
    for i, chunk in enumerate(chunks):
      all_chunks.append({
          'page_number': page_number,
          'chunk_index': i,
          'chunk_char_count': len(chunk),
          'chunk_word_count': len(chunk.split()),
          'chunk_token_count': len(chunk) / 4,
          "chunk_text":chunk

      })
  return all_chunks

In [ ]:
llm_chunked_pages = llm_based_chunk_pdf_pages(pages_and_texts, chunk_size=1000, model="gpt-4-1106-preview")

print(f"Total LLM-based chunks: {len(llm_chunked_pages)}")
print(f"First chunk (page {llm_chunked_pages[0]['page_number']}): {llm_chunked_pages[0]['chunk_text'][:200]}...")

LLM-based chunking pages:   0%|          | 0/1208 [00:00<?, ?it/s]

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## We will go ahead with fixed-size chunking

In [69]:
from spacy.lang.en import English

nlp = English()

nlp.add_pipe("sentencizer")

doc = nlp("This is a sentence. This is another sentence")

assert len(list(doc.sents)) == 2

list(doc.sents)

[This is a sentence., This is another sentence]

In [70]:
for item in tqdm(pages_and_texts):
  item['sentences'] = list(nlp(item['text']).sents)

  # Make sure all sentences are strings
  item['sentences'] = [str(sentence) for sentence in item['sentences']]

  # count the sentences
  item['page_sentence_count_spacy'] = len(item['sentences'])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [71]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_chart_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy,num_chunks
count,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.00,198.30,9.97,287.00,10.32,1.53
std,348.86,560.38,95.76,6.19,140.10,6.30,0.64
min,-41.00,0.00,1.00,1.00,0.00,0.00,0.00
25%,260.75,762.00,134.00,4.00,190.50,5.00,1.00
50%,562.50,1231.50,214.50,10.00,307.88,10.00,1.00
75%,864.25,1603.50,271.00,14.00,400.88,15.00,2.00
max,1166.00,2308.00,429.00,32.00,577.00,28.00,3.00


In [72]:
df

,page_number,page_chart_count,page_word_count,page_sentence_count_raw,page_token_count,text,sentences,page_sentence_count_spacy,sentence_chunks,num_chunks
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition,[Human Nutrition: 2020 Edition],1,[[Human Nutrition: 2020 Edition]],1
1,-40,0,1,1,0.00,,[],0,[],0
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...,[Human Nutrition: 2020 Edition UNIVERSITY OF...,1,[[Human Nutrition: 2020 Edition UNIVERSITY O...,1
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...,[Human Nutrition: 2020 Edition by University o...,1,[[Human Nutrition: 2020 Edition by University ...,1
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...,[Contents Preface University of Hawai‘i at M...,2,[[Contents Preface University of Hawai‘i at ...,1
...,...,...,...,...,...,...,...,...,...,...
1203,1162,1676,252,18,419.00,39. Exercise 10.2 & 11.3 reused “Egg Oval Food...,"[39., Exercise 10.2 & 11.3 reused “Egg Oval Fo...",18,"[[39., Exercise 10.2 & 11.3 reused “Egg Oval F...",2
1204,1163,1617,254,10,404.25,Images / Pixabay License; “Pumpkin Cartoon Ora...,[Images / Pixabay License; “Pumpkin Cartoon Or...,10,[[Images / Pixabay License; “Pumpkin Cartoon O...,1
1205,1164,1715,261,13,428.75,Flashcard Images Note: Most images in the fla...,[Flashcard Images Note: Most images in the fl...,13,[[Flashcard Images Note: Most images in the f...,2
1206,1165,1733,268,13,433.25,ShareAlike 11. Organs reused “Pancreas Organ ...,"[ShareAlike 11., Organs reused “Pancreas Orga...",13,"[[ShareAlike 11., Organs reused “Pancreas Org...",2


In [73]:
# define split size to turn groups of sentences into chunks

num_sentence_chunk_size = 10

def split_list(input_list: list, slice_size: int) -> list[list[str]]:
  return [input_list[i:i+slice_size] for i in range(0,len(input_list), slice_size)]

for item in tqdm(pages_and_texts):
  item['sentence_chunks'] = split_list(item['sentences'], num_sentence_chunk_size)

  item['num_chunks'] = len(item['sentence_chunks'])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [74]:
import random
random.sample(pages_and_texts,k=1)

[{'page_number': 569,
  'page_chart_count': 644,
  'page_word_count': 147,
  'page_sentence_count_raw': 15,
  'page_token_count': 161.0,
  'text': 'Food  Serving Niacin (mg) Percent Daily Value  Chicken  3 oz.  7.3  36.5  Tuna  3 oz.  8.6  43  Turkey  3 oz.  10.0  50  Salmon  3 oz.  8.5  42.5  Beef (90% lean)  3 oz.  4.4  22  Cereal (unfortified)  1 c.  5  25  Cereal (fortified)  1 c.  20  100  Peanuts  1 oz.  3.8  19  Whole wheat bread 1 slice  1.3  6.5  Coffee  8 oz.  0.5  2.5  Micronutrient Information Center: Niacin. Oregon State University,  Linus Pauling Institute. http://lpi.oregonstate.edu/mic/vitamins/ niacin. Updated in July 2013. Accessed October 22, 2017.  Pantothenic Acid (B5)  Figure 9.14 Pantothenic Acid‘s Role in the Citric Acid Cycle  Water-Soluble Vitamins  |  569',
  'sentences': ['Food  Serving Niacin (mg) Percent Daily Value  Chicken  3 oz.',
   ' 7.3  36.5  Tuna  3 oz.',
   ' 8.6  43  Turkey  3 oz.',
   ' 10.0  50  Salmon  3 oz.',
   ' 8.5  42.5  Beef (90% lean)  

In [75]:
import re

pages_and_chunks = []

for item in tqdm(pages_and_texts):
  for sentence_chunk in item['sentence_chunks']:
    chunk_dict = {}
    chunk_dict['page_number'] = item['page_number']

    joined_sentence_chunk = "".join(sentence_chunk).replace("  "," ").strip()
    joined_sentence_chunk = re.sub(r"\.([A-Z])", r'. \1', joined_sentence_chunk)

    chunk_dict['sentence_chunk'] = joined_sentence_chunk

    chunk_dict['chunk_char_count'] = len(joined_sentence_chunk)
    chunk_dict['chunk_word_count'] = len(joined_sentence_chunk.split())
    chunk_dict['chunk_token_count'] = len(joined_sentence_chunk) / 4

    pages_and_chunks.append(chunk_dict)

  0%|          | 0/1208 [00:00<?, ?it/s]

In [76]:
random.sample(pages_and_chunks, k=1)

[{'page_number': 629,
  'sentence_chunk': '1Based on DV = 1000 mg 2Based on DV = 1300 mg Source: Office of Dietary Supplements. (2020, March 26). Calcium. https://ods.od.nih.gov/factsheets/Calcium-HealthProfessional/ Titchenal, C. A., & Dobbs, J. (2007). A system to assess the quality of food sources of calcium. Journal of Food Composition and Analysis, 20(8), 717–724.doi: 10.1016/j.jfca.2006.04.013 \xa0 Learning Activities Technology Note: The second edition of the Human Nutrition Open Educational Resource (OER) textbook features interactive learning activities.\xa0 These activities are available in the web-based textbook and not available in the downloadable versions (EPUB, Digital PDF, Print_PDF, or Open Document). Learning activities may be used across various mobile devices, however, for the best user experience it is strongly recommended that users complete these activities using a desktop or laptop computer and in Google Chrome. \xa0 An interactive or media element has been excl

In [77]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,1843.00,1843.00,1843.00,1843.00
mean,583.38,734.44,112.56,183.61
std,347.79,447.54,71.26,111.89
min,-41.00,12.00,3.00,3.00
25%,280.50,315.00,45.00,78.75
50%,586.00,746.00,115.00,186.50
75%,890.00,1118.50,173.00,279.62
max,1166.00,1831.00,297.00,457.75


In [78]:
# random chunks with under 30 tokens in length
min_token_length = 30
for row in df[df['chunk_token_count'] <= min_token_length].sample(5).iterrows():
  print(f"Chunk token count: {row[1]['chunk_token_count']} | Text: {row[1]['sentence_chunk']}")

Chunk token count: 24.75 | Text: Table 9.33 Some Phytochemical’s Obtained from Diet and Their Related Functions 600 | Phytochemicals
Chunk token count: 23.25 | Text: They regulate osmolarity of the blood by regulating both solutes and The Urinary System | 115
Chunk token count: 23.5 | Text: view it online here: http://pressbooks.oer.hawaii.edu/ humannutrition2/?p=153   194 | Chloride
Chunk token count: 27.5 | Text: Figure 15.1 reused “Silohuette of Three People” by photo- nic.co.uk nic / Unsplash License 1158 | Attributions
Chunk token count: 11.0 | Text: Accessed October 5, 2017. Introduction | 433


In [79]:
pages_and_chunks_over_min_token_len = df[df['chunk_token_count'] > min_token_length].to_dict(orient='records')
pages_and_chunks_over_min_token_len[:2]

[{'page_number': -39,
  'sentence_chunk': 'Human Nutrition: 2020 Edition UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM ALAN TITCHENAL, SKYLAR HARA, NOEMI ARCEO CAACBAY, WILLIAM MEINKE-LAU, YA-YUN YANG, MARIE KAINOA FIALKOWSKI REVILLA, JENNIFER DRAPER, GEMADY LANGFELDER, CHERYL GIBBY, CHYNA NICOLE CHUN, AND ALLISON CALABRESE',
  'chunk_char_count': 308,
  'chunk_word_count': 42,
  'chunk_token_count': 77.0},
 {'page_number': -38,
  'sentence_chunk': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science and Human Nutrition Program is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.',
  'chunk_char_count': 210,
  'chunk_word_count': 30,
  'chunk_token_count': 52.5}]

In [80]:
import torch
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path = "all-mpnet-base-v2", device="cuda" if torch.cuda.is_available() else "cpu")

sentences = [
    "The Sentence Transformers library provides an easy and open-source way to create embeddings.",
    "Sentences can be embedded one by one or as a list of strings.",
    "Embeddings are one of the most powerful concepts in machine learning!",
    "Learn to use embeddings well and you'll be well on your way to being an AI engineer."
]

embeddings = embedding_model.encode(sentences)
embeddings_dict = dict(zip(sentences,embeddings))

for sentence, embedding in embeddings_dict.items():
  print(f"Sentence: {sentence}")
  print(f"Embedding: {embedding}")
  print()

Sentence: The Sentence Transformers library provides an easy and open-source way to create embeddings.
Embedding: [-1.99156404e-02  4.11679186e-02 -1.88775379e-02  6.57407343e-02
 -2.30733883e-02 -1.51006691e-02 -4.05872846e-03 -5.91936260e-02
  1.37239182e-02 -2.86808312e-02  3.22587788e-02  4.60161567e-02
 -5.52315414e-02  2.82665044e-02  3.58073004e-02 -4.94718216e-02
  4.60459702e-02 -1.20999757e-03 -1.34466961e-02  1.09683610e-02
  5.23573793e-02  4.38404344e-02  1.95452776e-02  5.30939326e-02
 -2.13442631e-02 -3.15952934e-02  4.08154493e-03 -2.61570159e-02
  4.53446135e-02  4.33328387e-04 -1.21952016e-02 -4.62402776e-03
  4.08579819e-02  1.90961622e-02  8.43360681e-07 -3.83117027e-03
 -2.27001458e-02  1.94588443e-03  8.51291884e-03  3.13563854e-03
  5.54310568e-02 -5.54998741e-02  1.08442185e-02  4.87202518e-02
 -3.79324518e-02 -9.72044887e-04  4.15215679e-02  2.44229995e-02
  9.37566534e-02  5.95447868e-02 -1.81962624e-02 -3.27963755e-02
  7.00938469e-03 -7.03648943e-03 -3.45217

In [81]:
%%time

for item in tqdm(pages_and_chunks_over_min_token_len):
  item['embedding'] = embedding_model.encode(item['sentence_chunk'])

  0%|          | 0/1680 [00:00<?, ?it/s]

CPU times: user 42.1 s, sys: 228 ms, total: 42.4 s
Wall time: 1min 14s


In [82]:
#Turn text chunks into a single list

text_chunks = [item['sentence_chunk'] for item in pages_and_chunks_over_min_token_len]

In [83]:
%%time
text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=32,
                                               convert_to_tensor=True)

text_chunk_embeddings

CPU times: user 14.4 s, sys: 77.1 ms, total: 14.4 s
Wall time: 25.1 s


tensor([[ 0.0674,  0.0902, -0.0051,  ..., -0.0221, -0.0232,  0.0126],
        [ 0.0552,  0.0592, -0.0166,  ..., -0.0120, -0.0103,  0.0227],
        [ 0.0280,  0.0340, -0.0206,  ..., -0.0054,  0.0213,  0.0313],
        ...,
        [ 0.0771,  0.0098, -0.0122,  ..., -0.0409, -0.0752, -0.0241],
        [ 0.1030, -0.0165,  0.0083,  ..., -0.0574, -0.0283, -0.0295],
        [ 0.0864, -0.0125, -0.0113,  ..., -0.0522, -0.0337, -0.0299]],
       device='cuda:0')

In [84]:
# save embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [85]:
# import saved file and view
text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embedding_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00,[ 6.74242675e-02 9.02281404e-02 -5.09548886e-...
1,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.50,[ 5.52156419e-02 5.92139773e-02 -1.66167244e-...
2,-37,Contents Preface University of Hawai‘i at Māno...,766,116,191.50,[ 2.79801842e-02 3.39813754e-02 -2.06426680e-...
3,-36,Lifestyles and Nutrition University of Hawai‘i...,941,144,235.25,[ 6.82566911e-02 3.81275006e-02 -8.46854132e-...
4,-35,The Cardiovascular System University of Hawai‘...,998,152,249.50,[ 3.30264494e-02 -8.49763490e-03 9.57159605e-...


In [86]:
import random

import torch
import numpy as np
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

text_chunks_and_embedding_df = pd.read_csv("text_chunks_and_embeddings_df.csv")
text_chunks_and_embedding_df['embedding'] = text_chunks_and_embedding_df['embedding'].apply(
    lambda x: np.fromstring(x.strip("[]"), sep=' ')
)

pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient='records')

embeddings = torch.tensor(np.array(text_chunks_and_embedding_df['embedding'].tolist()), dtype=torch.float32).to(device)
embeddings.shape

torch.Size([1680, 768])

In [87]:
text_chunks_and_embedding_df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00,"[0.0674242675, 0.0902281404, -0.00509548886, -..."
1,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.50,"[0.0552156419, 0.0592139773, -0.0166167244, -0..."
2,-37,Contents Preface University of Hawai‘i at Māno...,766,116,191.50,"[0.0279801842, 0.0339813754, -0.020642668, 0.0..."
3,-36,Lifestyles and Nutrition University of Hawai‘i...,941,144,235.25,"[0.0682566911, 0.0381275006, -0.00846854132, -..."
4,-35,The Cardiovascular System University of Hawai‘...,998,152,249.50,"[0.0330264494, -0.0084976349, 0.00957159605, -..."


In [29]:
embeddings[0]

tensor([ 6.7424e-02,  9.0228e-02, -5.0955e-03, -3.1755e-02,  7.3908e-02,
         3.5198e-02, -1.9799e-02,  4.6769e-02,  5.3573e-02,  5.0123e-03,
         3.3393e-02, -1.6222e-03,  1.7608e-02,  3.6265e-02, -3.1668e-04,
        -1.0712e-02,  1.5426e-02,  2.6218e-02,  2.7765e-03,  3.6494e-02,
        -4.4411e-02,  1.8936e-02,  4.9012e-02,  1.6402e-02, -4.8578e-02,
         3.1829e-03,  2.7299e-02, -2.0476e-03, -1.2283e-02, -7.2805e-02,
         1.2045e-02,  1.0730e-02,  2.1000e-03, -8.1777e-02,  2.6783e-06,
        -1.8143e-02, -1.2080e-02,  2.4717e-02, -6.2747e-02,  7.3544e-02,
         2.2162e-02, -3.2877e-02, -1.8010e-02,  2.2295e-02,  5.6137e-02,
         1.7951e-03,  5.2593e-02, -3.3174e-03, -8.3387e-03, -1.0628e-02,
         2.3192e-03, -2.2393e-02, -1.5301e-02, -9.9306e-03,  4.6532e-02,
         3.5747e-02, -2.5476e-02,  2.6369e-02,  3.7491e-03, -3.8268e-02,
         2.5833e-02,  4.1287e-02,  2.5818e-02,  3.3297e-02, -2.5178e-02,
         4.5152e-02,  4.4907e-04, -9.9662e-02,  4.9

In [88]:
from sentence_transformers import util

#Forming the Retrieval pipeline
query = "Macronutrients functions"

print(f"Query: {query}")

query_embedding = embedding_model.encode(query, convert_to_tensor=True)
query_embedding = query_embedding.to(embeddings.device)

from time import perf_counter as timer

start = timer()
dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]

end_time = timer()

print(f"Time take to get scores on {len(embeddings)} embeddings: {end_time - start:.5f} seconds")

top_results_dot_product = torch.topk(dot_scores, k=5)

top_results_dot_product

Query: Macronutrients functions
Time take to get scores on 1680 embeddings: 0.00035 seconds


torch.return_types.topk(
values=tensor([0.6926, 0.6738, 0.6646, 0.6536, 0.6473], device='cuda:0'),
indices=tensor([42, 47, 41, 51, 46], device='cuda:0'))

In [89]:
query_embedding.device

device(type='cuda', index=0)

In [90]:
embeddings

tensor([[ 0.0674,  0.0902, -0.0051,  ..., -0.0221, -0.0232,  0.0126],
        [ 0.0552,  0.0592, -0.0166,  ..., -0.0120, -0.0103,  0.0227],
        [ 0.0280,  0.0340, -0.0206,  ..., -0.0054,  0.0213,  0.0313],
        ...,
        [ 0.0771,  0.0098, -0.0122,  ..., -0.0409, -0.0752, -0.0241],
        [ 0.1030, -0.0165,  0.0083,  ..., -0.0574, -0.0283, -0.0295],
        [ 0.0864, -0.0125, -0.0113,  ..., -0.0522, -0.0337, -0.0299]],
       device='cuda:0')

In [91]:
import textwrap

def print_wrapped(text, wrap_length=80):
  wrapped_text = textwrap.fill(text, wrap_length)
  print(wrapped_text)

In [92]:
print(f"Query: {query}")
print("Results: ")

for score, idx in zip(top_results_dot_product[0], top_results_dot_product[1]):
  print(f"Score: {score:.4f}")

  print_wrapped(text_chunks_and_embedding_df.iloc[idx.item()]['sentence_chunk'])
  print(f"Page Number: {text_chunks_and_embedding_df.iloc[idx.item()]['page_number']}")
  print("\n")

Query: Macronutrients functions
Results: 
Score: 0.6926
Macronutrients Nutrients that are needed in large amounts are called
macronutrients. There are three classes of macronutrients: carbohydrates,
lipids, and proteins. These can be metabolically processed into cellular energy.
The energy from macronutrients comes from their chemical bonds. This chemical
energy is converted into cellular energy that is then utilized to perform work,
allowing our bodies to conduct their basic functions. A unit of measurement of
food energy is the calorie. On nutrition food labels the amount given for
“calories” is actually equivalent to each calorie multiplied by one thousand. A
kilocalorie (one thousand calories, denoted with a small “c”) is synonymous with
the “Calorie” (with a capital “C”) on nutrition food labels. Water is also a
macronutrient in the sense that you require a large amount of it, but unlike the
other macronutrients, it does not yield calories. Carbohydrates Carbohydrates
are molecule

In [93]:
text_chunks_and_embedding_df

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00,"[0.0674242675, 0.0902281404, -0.00509548886, -..."
1,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.50,"[0.0552156419, 0.0592139773, -0.0166167244, -0..."
2,-37,Contents Preface University of Hawai‘i at Māno...,766,116,191.50,"[0.0279801842, 0.0339813754, -0.020642668, 0.0..."
3,-36,Lifestyles and Nutrition University of Hawai‘i...,941,144,235.25,"[0.0682566911, 0.0381275006, -0.00846854132, -..."
4,-35,The Cardiovascular System University of Hawai‘...,998,152,249.50,"[0.0330264494, -0.0084976349, 0.00957159605, -..."
...,...,...,...,...,...,...
1675,1164,Flashcard Images Note: Most images in the flas...,1305,186,326.25,"[0.0185622536, -0.0164277665, -0.0127045633, -..."
1676,1164,Hazard Analysis Critical Control Points reused...,375,51,93.75,"[0.0334720612, -0.0570440851, 0.0151489386, -0..."
1677,1165,ShareAlike 11. Organs reused “Pancreas Organ A...,1286,175,321.50,"[0.0770515501, 0.00978557579, -0.0121817412, 0..."
1678,1165,Sucrose reused “Figure 03 02 05” by OpenStax B...,410,63,102.50,"[0.103045158, -0.0164701864, 0.00826846063, 0...."


In [94]:
def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer=embedding_model,
                                n_resources_to_return: int=5,
                                print_time: bool=True):

  query_embedding = model.encode(query, convert_to_tensor=True)
  start_time = timer()
  dot_scores = util.dot_score(query_embedding, embeddings)[0]
  end_time = timer()

  if print_time:
    print(f"[INFO] Time taken to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds")

  scores, indices = torch.topk(input=dot_scores,
                               k=n_resources_to_return)

  return scores, indices

def print_top_results_and_scores(query: str,
                                 embeddings: torch.tensor,
                                 pages_and_chunks: list[dict]=pages_and_chunks,
                                 n_resources_to_return: int =5):

  scores, indices = retrieve_relevant_resources(query, embeddings, n_resources_to_return = n_resources_to_return)

  print(f"Query: {query}\n")
  print("Results:")
  for score, index in zip(scores, indices):
    print(f"Score: {score:.4f}")
    print_wrapped(pages_and_chunks[index]['sentence_chunk'])
    print(f"Page number: {pages_and_chunks[index]['page_number']}")
    print("\n")

In [95]:
#Testing the function

query = "symptoms of pellagra"

print_top_results_and_scores(query, embeddings)

[INFO] Time taken to get scores on 1680 embeddings: 0.00016 seconds
Query: symptoms of pellagra

Results:
Score: 0.5000
Niacin deficiency is commonly known as pellagra and the symptoms include
fatigue, decreased appetite, and indigestion.  These symptoms are then commonly
followed by the four D’s: diarrhea, dermatitis, dementia, and sometimes death.
Figure 9.12  Conversion of Tryptophan to Niacin Water-Soluble Vitamins | 565
Page number: 565


Score: 0.3741
car. Does it drive faster with a half-tank of gas or a full one?It does not
matter; the car drives just as fast as long as it has gas. Similarly, depletion
of B vitamins will cause problems in energy metabolism, but having more than is
required to run metabolism does not speed it up. Buyers of B-vitamin supplements
beware; B vitamins are not stored in the body and all excess will be flushed
down the toilet along with the extra money spent. B vitamins are naturally
present in numerous foods, and many other foods are enriched with the

In [96]:
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes/ (2**30))

print(f"Available GPU Memory: {gpu_memory_gb} GB")

Available GPU Memory: 15 GB


In [97]:
# Note: The following is Gemma-focused, however, there are more and more
# LLMs of the 2B and 7B size appearing for local use.

if gpu_memory_gb < 5.1:
    print(f"Your available GPU memory is {gpu_memory_gb}GB — you may not have enough "
          "memory to run a Gemma LLM locally without quantization.")
elif gpu_memory_gb < 8.1:
    print(f"GPU memory: {gpu_memory_gb}GB | Recommended model: Gemma 2B in 4-bit precision.")
    use_quantization_config = True
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb < 19.0:
    print(f"GPU memory: {gpu_memory_gb}GB | Recommended model: Gemma 2B in float16 "
          "or Gemma 7B in 4-bit precision.")
    use_quantization_config = False
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb >= 19.0:
    print(f"GPU memory: {gpu_memory_gb}GB | Recommended model: Gemma 7B in 4-bit "
          "or float16 precision.")
    use_quantization_config = False
    model_id = "google/gemma-7b-it"

print(f"use_quantization_config set to: {use_quantization_config}")
print(f"model_id set to: {model_id}")

GPU memory: 15GB | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.
use_quantization_config set to: False
model_id set to: google/gemma-2b-it


In [98]:
from huggingface_hub import login
from google.colab import userdata

login(token=userdata.get("HF_TOKEN"))

In [41]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available


from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16)


attn_implementation = "sdpa"

print(f"[INFO] Using attention implementation: {attn_implementation}")

model_id = model_id
print(f"[INFO] Using model_id: {model_id}")

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)

llm_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id,
                                                 torch_dtype=torch.float16,
                                                 quantization_config = quantization_config if use_quantization_config else None,
                                                 low_cpu_mem_usage = False,
                                                 attn_implementation= attn_implementation)

if not use_quantization_config:
  llm_model.to("cuda")

[INFO] Using attention implementation: sdpa
[INFO] Using model_id: google/gemma-2b-it


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [42]:
llm_model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): GemmaRMSNorm((2048,), 

In [43]:
sum([param.numel() for param in llm_model.parameters()])

2506172416

In [99]:
input_text = "What are the macronutrients, and what roles do they play in the human body?"
print(f"Input text:\n{input_text}")

dialogue_template = [
    {"role":"user",
    "content":input_text}
]

prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                       tokenize=False,
                                       add_generation_prompt=True)

print(f"\nPrompt (formatted):\n{prompt}")

Input text:
What are the macronutrients, and what roles do they play in the human body?

Prompt (formatted):
<bos><start_of_turn>user
What are the macronutrients, and what roles do they play in the human body?<end_of_turn>
<start_of_turn>model



In [100]:
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
print(f"MOdel input (tokenized):\n{input_ids}\n")

outputs = llm_model.generate(**input_ids,
                             max_new_tokens=256)

print(f"Modle output: \n{outputs[0]}")

MOdel input (tokenized):
{'input_ids': tensor([[     2,      2,    106,   1645,    108,   1841,    708,    573, 186809,
         184592, 235269,    578,   1212,  16065,    749,    984,   1554,    575,
            573,   3515,   2971, 235336,    107,    108,    106,   2516,    108]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}

Modle output: 
tensor([     2,      2,    106,   1645,    108,   1841,    708,    573, 186809,
        184592, 235269,    578,   1212,  16065,    749,    984,   1554,    575,
           573,   3515,   2971, 235336,    107,    108,    106,   2516,    108,
         21404, 235269,   1517, 235303, 235256,    476,  25497,    576,    573,
        186809, 184592,    578,   1024,  16065,    575,    573,   3515,   2971,
        235292,    109,    688,  12298,   1695, 184592,  66058,    109, 235287,
          5231, 156615,  56227,  66058,    108,    141, 23

In [101]:
print(tokenizer.decode(outputs[0]))

<bos><bos><start_of_turn>user
What are the macronutrients, and what roles do they play in the human body?<end_of_turn>
<start_of_turn>model
Sure, here's a breakdown of the macronutrients and their roles in the human body:

**Macronutrients:**

* **Carbohydrates:**
    * Provide energy for the body's cells and tissues.
    * Carbohydrates are the primary source of energy for most cells.
    * Complex carbohydrates are those that take longer to digest, such as whole grains, fruits, and vegetables.
    * Simple carbohydrates are those that are quickly digested, such as sugar, starch, and lactose.

* **Proteins:**
    * Build and repair tissues, enzymes, and hormones.
    * Proteins are essential for immune function, hormone production, and tissue repair.
    * There are different types of proteins, each with specific functions.

* **Fats:**
    * Provide energy, insulation, and help absorb vitamins.
    * Healthy fats include olive oil, avocado, nuts, and seeds.
    * Trans fats can raise

In [102]:
# Nutrition-style questions generated with GPT-4
gpt4_questions = [
    "What are the macronutrients, and what roles do they play in the human body?",
    "How do vitamins and minerals differ in their roles and importance for health?",
    "Describe the process of digestion and absorption of nutrients in the human body.",
    "What role does fibre play in digestion? Name five fibre containing foods.",
    "Explain the concept of energy balance and its importance in weight management."
]

# Manually created question list
manual_questions = [
    "How often should infants be breastfed?",
    "What are symptoms of pellagra?",
    "How does saliva help with digestion?",
    "What is the RDI for protein per day?",
    "Water soluble vitamins"
]

# Combine both lists
query_list = gpt4_questions + manual_questions



In [103]:
import random
query = random.choice(query_list)

print(f"Query: {query}")

scores, indices = retrieve_relevant_resources(query, embeddings)

scores, indices

Query: What is the RDI for protein per day?
[INFO] Time taken to get scores on 1680 embeddings: 0.00010 seconds


(tensor([0.6793, 0.6771, 0.6418, 0.6109, 0.5908], device='cuda:0'),
 tensor([ 609,  607, 1007,  608, 1005], device='cuda:0'))

In [104]:
def prompt_formatter(query: str, context_items: list[dict]) -> str:
  """
  Augments query with text-based context from context_items.
  """

  # Join context items into one dotted paragraph
  context = "* " + "\n* ".join([item["sentence_chunk"] for item in context_items])

  # Create a base prompt with examples to help the model
  # Note: this is very customizable; we’re using 3 examples of the desired answer style.
  base_prompt = """Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.

Example 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet.

Example 2:
Query: What are the causes of type 2 diabetes?
Answer: Type 2 diabetes is often associated with overnutrition, particularly the overconsumption of calories leading to obesity. Factors such as genetics and physical inactivity also play roles.

Example 3:
Query: What is the importance of hydration for physical performance?
Answer: Hydration is crucial for physical performance because water plays key roles in maintaining blood volume, regulating body temperature, and lubricating joints.

Now use the following context items to answer the user query:
{context}

The context contains the answer. Look carefully and extract relevant information.

Relevant passages: <extract relevant passages from the context here>

User query: {query}
Answer:"""

  base_prompt = base_prompt.format(context=context, query=query)

  dialogue_template = [
      {"role":"user",
       "content":base_prompt}
  ]

  prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                         tokenize=False,
                                         add_generation_prompt=True)

  return prompt

In [105]:
query = random.choice(query_list)

print(f"Query: {query}")

scores, indices = retrieve_relevant_resources(query, embeddings)

context_items = [pages_and_chunks[i] for i in indices]

prompt = prompt_formatter(query=query, context_items= context_items)

print(prompt)

Query: Explain the concept of energy balance and its importance in weight management.
[INFO] Time taken to get scores on 1680 embeddings: 0.00010 seconds
<bos><start_of_turn>user
Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.

Example 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet.

Example 2:
Query: What are the causes of type 2 diabetes?
Answer: Type 2 diabetes is often associated with overnutrition, particularly the overconsumption of calories leading to obesity. Factors such as genetics and physical inactivity also play roles.

Example 3:
Query: What i

In [106]:
def ask(query,
        temperature=0.7,
        max_new_tokens=512,
        format_answer_text=True,
        return_answer_only=True):

  scores, indices = retrieve_relevant_resources(query, embeddings)

  context_items = [pages_and_chunks[i] for i in indices]

  for i, item in enumerate(context_items):
    item['score'] = scores[i].cpu()

  prompt = prompt_formatter(query, context_items)

  input_ids = tokenizer(prompt, return_tensors='pt').to('cuda')

  outputs = llm_model.generate(**input_ids,
                               temperature=temperature,
                               do_sample=True,
                               max_new_tokens=max_new_tokens)

  output_text = tokenizer.decode(outputs[0])

  if format_answer_text:
    output_text = output_text.replace(prompt,"").replace("<bos>","").replace("<eos>","")

  if return_answer_only:
    return output_text

  return output_text, context_items

In [107]:
query = random.choice(query_list)
print("Query: ",query)

answer, context_items = ask(query=query,
                            temperature=0.7,
                            max_new_tokens=512,
                            return_answer_only=False)

print("Answer:\n")
print_wrapped(answer)
# print("Context Items:")
# context_items

Query:  What are the macronutrients, and what roles do they play in the human body?
[INFO] Time taken to get scores on 1680 embeddings: 0.00015 seconds
Answer:

Sure, here are the relevant passages from the context:  * Nutrients are
substances required by the body to perform its basic functions. * Macronutrients
are nutrients needed in large amounts. They are divided into three classes:
carbohydrates, lipids, and proteins. * Carbohydrates are molecules composed of
carbon, hydrogen, and oxygen. They are the body's main source of energy. *
Lipids are molecules composed of carbon, hydrogen, and oxygen. They are used for
energy and to build and repair tissues. * Proteins are macromolecules composed
of chains of subunits called amino acids. They provide structure and function to
cells. * Water is a macronutrient that is crucial for life's basic processes. It
is essential for hydration, temperature regulation, and chemical reactions.


## Evaluation of RAG

In [53]:
!pip install -q ragas datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.0/221.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 80.1 

In [108]:
import pandas as pd
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    context_precision,
    context_recall,
    answer_relevancy,
    faithfulness
)

try:
  from ragas.metrics import context_entity_recall

except ImportError:
  context_entity_recall = None
  print("context_entity_recall not available in this version")


try:
  from ragas.metrics import noise_robustness

except ImportError:
  noise_robustness = None
  print("noise robustness not available in this version")


noise robustness not available in this version


In [109]:
# Define evaluation questions and ground truth answers
eval_questions = [
    "How often should infants be breastfed?",
    "What are symptoms of pellagra?",
    "How does saliva help with digestion?",
    "What is the recommended protein intake per day, based on your weight?",
    "What are micronutrients?"
]

ground_truth_answers = [
    "A newborn infant (birth to 28 days) requires feedings eight to twelve times a day or more. Between 1 and 3 months of age, the breastfed infant becomes more efficient, and the number offeedings per day often become fewer even though the amount ofmilk consumed stays the same. ",

    "iacin deficiency is commonly known as pellagra and the symptoms include fatigue, decreased appetite, and indigestion. These symptoms are then commonly followed by the four D’s: diarrhea, dermatitis, dementia, and sometimes deat",

    "The mechanical and chemical digestion of carbohydrates begins in the mouth. Chewing, also known as mastication, crumbles the carbohydrate foods into smaller and smaller pieces. The salivary glands in the oral cavity secrete saliva that coats the food particles. Saliva contains the enzyme, salivary amylase. This enzyme breaks the bonds between the monomeric sugar units of disaccharides, oligosaccharides, and starches. The salivary amylase breaks down amylose and amylopectin into smaller chains of glucose, called dextrins and maltose. The increased concentration of maltose in the mouth that results from the mechanical and chemical breakdown of starches in whole grains is what enhances their sweetness. Only about five percent of starches are broken down in the mouth. (This is a good thing as more glucose in the mouth would lead to more tooth decay.) When carbohydrates reach the stomach no further chemical breakdown occurs because the amylase enzyme does not function in the acidic conditions of the stomach. ",

    "your exact recommended protein intake per day based on your weight by using the following equation: (Weight in lbs. ÷ 2.2 lb/kg) × 0.8 g/kg Note that if a person is overweight, the amount of dietary protein recommended can be overestimated. ",

    "Micronutrients are nutrients required by the body in lesser amounts, but are still essential for carrying out bodily functions. Micronutrients include all the essential minerals and vitamins. "
]


In [110]:
def generate_rag_answer(query):
  scores, indices = retrieve_relevant_resources(query=query, embeddings=embeddings)

  context_items = [pages_and_chunks[i] for i in indices]

  prompt = prompt_formatter(query=query, context_items=context_items)

  input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
  outputs = llm_model.generate(**input_ids,
                               temperature=0.7,
                               do_sample=True,
                               max_new_tokens=256)

  output_text = tokenizer.decode(outputs[0])
  answer = output_text.replace(prompt,"").strip()

  contexts = [item['sentence_chunk'] for item in context_items]

  return answer, contexts

In [111]:
evaluation_data = []

print("Generating RAG answers for evaluation...")
for question, ground_truth in zip(eval_questions, ground_truth_answers):
  print(f"Processing: {question[:50]}...")

  rag_answer, contexts = generate_rag_answer(question)

  evaluation_data.append({
      "question":question,
      "answer":rag_answer,
      "contexts":contexts,
      "ground_truth":ground_truth
  })

Generating RAG answers for evaluation...
Processing: How often should infants be breastfed?...
[INFO] Time taken to get scores on 1680 embeddings: 0.00011 seconds
Processing: What are symptoms of pellagra?...
[INFO] Time taken to get scores on 1680 embeddings: 0.00011 seconds
Processing: How does saliva help with digestion?...
[INFO] Time taken to get scores on 1680 embeddings: 0.00010 seconds
Processing: What is the recommended protein intake per day, ba...
[INFO] Time taken to get scores on 1680 embeddings: 0.00014 seconds
Processing: What are micronutrients?...
[INFO] Time taken to get scores on 1680 embeddings: 0.00102 seconds


In [112]:
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")

In [113]:
from ragas.llms import LangchainLLMWrapper  # Updated way
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

judge_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", temperature=0))

# Embedding model: text-embedding-3-small
judge_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

/tmp/ipython-input-2504356593.py:5: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  judge_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", temperature=0))
/tmp/ipython-input-2504356593.py:8: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  judge_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))


In [114]:
eval_dataset = Dataset.from_pandas(pd.DataFrame(evaluation_data))

metrics = [
    context_precision,
    context_recall,
    answer_relevancy,
    faithfulness
]

if context_entity_recall is not None:
  metrics.append(context_entity_recall)

if noise_robustness is not None:
  metrics.append(noise_robustness)

print("Running RAGAS evaluation..")

results = evaluate(
    dataset = eval_dataset,
    metrics = metrics,
    llm=judge_llm,
    embeddings=judge_embeddings
)

results_df = results.to_pandas()





Running RAGAS evaluation..


Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

In [115]:
results_df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,answer_relevancy,faithfulness,context_entity_recall
0,How often should infants be breastfed?,[milk is the best source to fulfill nutritiona...,<bos>The context does not specify how often in...,A newborn infant (birth to 28 days) requires f...,1.000000,1.0,0.000000,0.0,0.500000
1,What are symptoms of pellagra?,[Niacin deficiency is commonly known as pellag...,"<bos>Sure, here are the relevant passages from...",iacin deficiency is commonly known as pellagra...,1.000000,1.0,0.988674,1.0,0.111111
2,How does saliva help with digestion?,[Digestion and Absorption of Carbohydrates UNI...,<bos>The passage does not provide any informat...,The mechanical and chemical digestion of carbo...,1.000000,0.9,0.000000,0.0,0.600000
3,What is the recommended protein intake per day...,[or when reducing energy intake.5 See Table 16...,<bos>The context does not specify the recommen...,your exact recommended protein intake per day ...,0.250000,0.0,0.000000,0.0,0.000000
4,What are micronutrients?,[Water There is one other nutrient that we mus...,<bos>The context does not provide any relevant...,Micronutrients are nutrients required by the b...,0.755556,1.0,0.000000,0.0,0.333333


In [118]:
metric_cols = []
for col in results_df.columns:
  if col in results_df.columns:
    if col not in ['user_input','retrieved_contexts','response','reference']:
      try:
        numeric_values = pd.to_numeric(results_df[col],errors='coerce')
        if not numeric_values.isna().all():
          metric_cols.append(col)
      except:
        pass

# Create a clean DataFrame for display
clean_results = pd.DataFrame()
clean_results['Question'] = [
    f"Q{i+1}: {q[:50]}{'...' if len(q) > 50 else ''}"
    for i, q in enumerate(results_df['user_input'])
]

# Add metric scores
for col in metric_cols:
    clean_results[col.replace('_', ' ').title()] = results_df[col].round(3)

# Display the table with better formatting
print(clean_results.to_string(index=False, float_format='%.3f'))

avg_scores = {}

print(f"\n📊 PERFORMANCE SUMMARY")
print("-" * 50)

for col in metric_cols:
    avg_score = results_df[col].mean()
    avg_scores[col] = avg_score

    # Performance indicator
    if avg_score >= 0.8:
        indicator = "Excellent"
    elif avg_score >= 0.6:
        indicator = "Good"
    elif avg_score >= 0.4:
        indicator = "Fair"
    else:
        indicator = "Poor"

    # Display formatted output
    print(f"{col.replace('_', ' ').title():<25}: {avg_score:.3f}   {indicator}")



                                                 Question  Context Precision  Context Recall  Answer Relevancy  Faithfulness  Context Entity Recall
               Q1: How often should infants be breastfed?              1.000           1.000             0.000         0.000                  0.500
                       Q2: What are symptoms of pellagra?              1.000           1.000             0.989         1.000                  0.111
                 Q3: How does saliva help with digestion?              1.000           0.900             0.000         0.000                  0.600
Q4: What is the recommended protein intake per day, ba...              0.250           0.000             0.000         0.000                  0.000
                             Q5: What are micronutrients?              0.756           1.000             0.000         0.000                  0.333

📊 PERFORMANCE SUMMARY
--------------------------------------------------
Context Precision        : 0.801   Exc

In [119]:
# Summarize performance categories
excellent = sum(1 for score in avg_scores.values() if score >= 0.8)
good = sum(1 for score in avg_scores.values() if 0.6 <= score < 0.8)
fair = sum(1 for score in avg_scores.values() if 0.4 <= score < 0.6)
poor = sum(1 for score in avg_scores.values() if score < 0.4)

print(f"\n⭐ Excellent metrics: {excellent}")
print(f"👍 Good metrics: {good}")
print(f"👌 Fair metrics: {fair}")
print(f"⚠️ Poor metrics: {poor}")

# Key insights
print(f"\n🔍 KEY INSIGHTS")
print("-" * 50)

best_metric = max(avg_scores, key=avg_scores.get)
worst_metric = min(avg_scores, key=avg_scores.get)

print(f"✅ Best performing: {best_metric.replace('_', ' ').title()} ({avg_scores[best_metric]:.3f})")
print(f"❌ Needs improvement: {worst_metric.replace('_', ' ').title()} ({avg_scores[worst_metric]:.3f})")

# Specific recommendations based on scores
if avg_scores.get('context_precision', 0) > 0.7 and avg_scores.get('answer_relevancy', 0) < 0.3:
    print("\n⚠️ Issue detected: Good retrieval but poor answer generation")
    print("➡️ Fix your prompt template to better use retrieved context")

if avg_scores.get('context_recall', 0) < 0.5:
    print("\n⚠️ Issue detected: Poor context retrieval")
    print("➡️ Improve your embedding model or chunk strategy")

if avg_scores.get('answer_faithfulness', 0) < 0.6:
    print("\n⚠️ Issue detected: Answers may contain hallucinations")
    print("➡️ Consider better grounding or verification steps")

# Final summary line
print("\n📁 Results saved to 'rag_evaluation_results.csv'")


detailed_results = pd.DataFrame(evaluation_data)
for col in metric_cols:
  detailed_results[col] = results_df[col].values

detailed_results.to_csv("rag_evaluation_results.csv",index=False)



⭐ Excellent metrics: 1
👍 Good metrics: 1
👌 Fair metrics: 0
⚠️ Poor metrics: 3

🔍 KEY INSIGHTS
--------------------------------------------------
✅ Best performing: Context Precision (0.801)
❌ Needs improvement: Answer Relevancy (0.198)

⚠️ Issue detected: Good retrieval but poor answer generation
➡️ Fix your prompt template to better use retrieved context

⚠️ Issue detected: Answers may contain hallucinations
➡️ Consider better grounding or verification steps

📁 Results saved to 'rag_evaluation_results.csv'
